### 第三课 机器学习构建chatbot

* 主要内容
  * 关于聊天机器人的思考
    * 1.工程考量
    * 2.机器学习角度考量
   * 预备知识
     * 1.检索与匹配
     * 2.分类与朴素贝叶斯
   * chatterbot
     * 1.架构与使用方法
     * 2.源码分析


### 传统聊天机器人
* NLP基础知识
  * 基本分词
  * 关键词抽取(TF-IDF等)
  * 正则表达式模式匹配
  * ....
* Machine Learning相关知识
  * 文本表示与匹配
  * 分类(文本场景分析)
  * 数据驱动(特征工程)

### 检索与匹配
* 基于检索与匹配
  * 知识库(存储了问题与回复内容)
  * 检索：搜寻相关问题
  * 匹配：对结果进行排序

开放域机器人：
* 什么都想插一脚

Closet Match Adapter
* 字符串模糊匹配（编辑距离）
  * 编辑距离/Levenshtein距离，是指两个字符串之间，由一个转成另外一个所需要的最少编辑操作次数


```
# !pip install python-Levenshtein
import Levenshtein
texta=u'七月在线'
textb=u'七天在线'
Levenshtein.distance(texta,textb)
```


  * 计算出两个句子之间的置信度
    * 用来评估你提出的问题和数据库中存在的问题的相似程度。如果自己提出的问题和数据库中存在的问题**编辑距离**很近，那么可以认为是同一个问题，这样就可以根据数据库中已有的问题反馈给用户数据库中已存在的答案


* TF-IDF
  * 如果使用上面提到的编辑距离，这时候会出现一个问题，因为在评估中每个词我们设定的权重是一样的，比如下面两句话，虽然编辑距离很相似，但是提问的不是一个问题，面对这种情况我们可以用TF-IDF进行根据不同的权重，提取出关键词，衡量一个词对整个句子的重要性，针对关键词来做比对，这样就可以区分出不是一个问题
    * 你喜欢什么书
    * 你喜欢什么电影

* Closest Meaning Adapater
  * 借助nltk的WordNet，近义词评估。如下面两个句子，从nltk巨大的近义词库中我们发现关键词是近义词，那么就可以认为这两句话表达含义是一样的
    * 你喜欢这本书吗
    * 你喜欢这本资料吗

* Time Logic Adapter
  * 处理涉及时间的提问。这一块范围就设置的比较具体了，主要回答针对时间的提问

* Mathematical Evaluation Adapter
  * 涉及数学运算。


* 场景分类
  * 朴素贝叶斯(NB)
    * 我们输入一个词(篮球鞋)，会根据概率去判断它属于哪个类(运动鞋)
    $$P(B \mid A)=\frac{P(A \mid B) P(B)}{P(A)}$$

聊天机器人遇到的第一个问题就是用什么方式去回答这个问题，会涉及两个问题：
* **匹配的时候用什么方式**
* **不同的场景用什么方式**

### Chatterbot聊天机器人

ChatterBot是一个基于机器学习的聊天机器人引擎，构建在python上，主要特点是可以从已有的对话中进行学习(记忆匹配)

* 官网：https://chatterbot.readthedocs.io/en/stable/
* Github:https://github.com/gunthercox/ChatterBot

每个部分都设计了不同的"适配器"(Adapter)
* **机器人应答逻辑** => `Logic Adapters`
  * Closest Match Adapter
    * 字符串模糊匹配(**编辑距离**)
  * Closest Meaning Adapter
    * 借助nltk的WordNet，**近义词评估**
  * Time Logic Adapter
    * 处理涉及**时间**的提问
  * Mathematical Evaluation Adapter
    * 涉及**数学运算**

  * **存储器后端** => `Storage Adapters`
    * Read Only Mode
      * 只读模式，当有输入数据到chatterbot的时候，数
据库并不会发生改变
    * Json Database Adapter
      * 用以存储对话数据的接口，对话数据以Json格式
进行存储。线上不建议采取这种方法，需要缓存
    * Mongo Database Adapter
      * 以MongoDB database方式来存储对话数据

  * **输入形式** => `Input Adapters`
    * Variable input type adapter
      * 允许chatter bot接收不同类型的输入的，如
strings,dictionaries和Statements
    * Terminal adapter
      * 使得ChatterBot可以通过终端进行对话
    * HipChat Adapter
      * 使得ChatterBot 可以从HipChat聊天室获取输入语
句，通过HipChat 和 ChatterBot 进行对话
    * Speech recognition
      * 语音识别输入，详见chatterbot-voice 

  * **输出形式** => `Output Adapters`
    * Output format adapter
      * 支持text，json和object格式的输出
    * Terminal adapter
    * HipChat Adapter
    * Mailgun adapter
      * 允许chat bot基于Mailgun API进行邮件的发送
    * Speech synthesis
      * TTS(Text to speech)部分，详见chatterbot-voice 

### 代码

chatterbot是一款python接口的，基于一系列规则和机器学习算法完成的聊天机器人。具有结构清晰，可扩展性好，简单实用的特点。

##### 安装
是的，安装超级简单，用pip就可以啦

 pip install chatterbot

#### 各式各样的Adapter

大家已经知道chatterbot的聊天逻辑和输入输出以及存储，是由各种adapter来限定的，我们先看看流程图，一会再看例子，看看怎么用
![](http://chatterbot.readthedocs.io/en/stable/_images/chatterbot-process-flow.svg)

### 基础版本

#### 基于编辑距离

In [10]:
# -*- coding: utf-8 -*-
# !pip install chatterbot
# https://github.com/gunthercox/ChatterBot/blob/master/examples/default_response_example.py
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

# 构建ChatBot并指定Adapter
bot = ChatBot(
    'Default Response Example Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',#JsonFileStorageAdapter was removed.
    # 基于编辑距离
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',#编辑距离
             'default_response': 'I am sorry, but I do not understand.',#阈值低于0.65返回值
            # 'import_path': 'chatterbot.logic.LowConfidenceAdapter',  was removed
            'maximum_similarity_threshold': 0.65,
        }
    ]
)
trainer=ListTrainer(bot)#ListTreainer表示列表，两个人对话

# 手动给定一点语料用于训练
trainer.train([
    'How can I help you?',
    'I want to create a chat bot',
    'Have you read the documentation?',
    'No, I have not',
    'This should help get you started: http://chatterbot.rtfd.org/en/latest/quickstart.html'
])

# 给定问题并取回结果
question = 'How do I make an omelette?'
print('question:',question)
response = bot.get_response(question)
print('ans:',response)

print("\n")
question = 'how to make a chat bot?'
print('question:',question)
response = bot.get_response(question)
print('ans:',response)

List Trainer: [####################] 100%
question: How do I make an omelette?
ans: I am sorry, but I do not understand.


question: how to make a chat bot?
ans: I am sorry, but I do not understand.


#### 处理时间和数学计算的Adapter

In [11]:
# -*- coding: utf-8 -*-
# https://github.com/gunthercox/ChatterBot
from chatterbot import ChatBot


bot = ChatBot(
    "Math & Time Bot",
    logic_adapters=[
        "chatterbot.logic.MathematicalEvaluation",#处理时间
        "chatterbot.logic.TimeLogicAdapter"#处理数学计算
    ],
    input_adapter="chatterbot.input.VariableInputTypeAdapter",
    output_adapter="chatterbot.output.OutputAdapter"
)

# 进行数学计算
question = "What is 4 + 9?"
print(question)
response = bot.get_response(question)
print(response)

print("\n")

# 回答和时间相关的问题
question = "What time is it?"
print(question)
response = bot.get_response(question)
print(response)

What is 4 + 9?
4 + 9 = 13


What time is it?
The current time is 10:25 AM


#### 导出语料到json文件

In [13]:
# -*- coding: utf-8 -*-
# https://github.com/zhangyuankai2018/chatterbot-corpus
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
'''
如果一个已经训练好的chatbot，你想取出它的语料，用于别的chatbot构建，可以这么做
'''

chatbot = ChatBot(
    'Export Example Bot',
)
#trainer='chatterbot.trainers.ChatterBotCorpusTrainer'
trainer=ChatterBotCorpusTrainer(chatbot)

# 训练一下咯
trainer.train('/content/drive/My Drive/VQA/english')

# 把语料导出到json文件中
trainer.export_for_training('/content/drive/My Drive/VQA/my_export.json')

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


#### 使用Ubuntu数据集构建聊天机器人

In [16]:
from chatterbot import ChatBot
import logging
from chatterbot.trainers import UbuntuCorpusTrainer

'''
这是一个使用Ubuntu语料构建聊天机器人的例子
'''

# 允许打日志
logging.basicConfig(level=logging.INFO)

chatbot = ChatBot(
    'Example Bot',
)
trainer=UbuntuCorpusTrainer(chatbot)

# 使用Ubuntu数据集开始训练
trainer.train()

# 我们来看看训练后的机器人的应答
response = chatbot.get_response('How are you doing today?')
print(response)

#### 借助微软的聊天机器人

In [18]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from settings import Microsoft

'''
关于获取微软的user access token请参考以下的文档
https://docs.botframework.com/en-us/restapi/directline/
'''

chatbot = ChatBot(
    'MicrosoftBot',
    directline_host = Microsoft['directline_host'],
    direct_line_token_or_secret = Microsoft['direct_line_token_or_secret'],
    conversation_id = Microsoft['conversation_id'],
    input_adapter='chatterbot.input.Microsoft',
    output_adapter='chatterbot.output.Microsoft',
    trainer='chatterbot.trainers.ChatterBotCorpusTrainer'
)

chatbot.train('/content/sample_data/english')

# 是的，会一直聊下去
while True:
    try:
        response = chatbot.get_response(None)

    # 直到按ctrl-c 或者 ctrl-d 才会退出
    except (KeyboardInterrupt, EOFError, SystemExit):
        break 

#### HipChat聊天室Adapter

In [19]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from settings import HIPCHAT

'''
炫酷一点，你可以接到一个HipChat聊天室，你需要一个user token，下面文档会告诉你怎么做
https://developer.atlassian.com/hipchat/guide/hipchat-rest-api/api-access-tokens
'''

chatbot = ChatBot(
    'HipChatBot',
    hipchat_host=HIPCHAT['HOST'],
    hipchat_room=HIPCHAT['ROOM'],
    hipchat_access_token=HIPCHAT['ACCESS_TOKEN'],
    input_adapter='chatterbot.input.HipChat',
    output_adapter='chatterbot.output.HipChat',
    trainer='chatterbot.trainers.ChatterBotCorpusTrainer'
)

chatbot.train('/content/sample_data/english')

# 没错，while True，会一直聊下去！
while True:
    try:
        response = chatbot.get_response(None)

    # 直到按ctrl-c 或者 ctrl-d 才会退出
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

#### 邮件回复的聊天系统

In [ ]:
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from settings import MAILGUN

'''
这个功能需要你新建一个文件settings.py，并在里面写入如下的配置:
MAILGUN = {
    "CONSUMER_KEY": "my-mailgun-api-key",
    "API_ENDPOINT": "https://api.mailgun.net/v3/my-domain.com/messages"
}
'''

# 下面这个部分可以改成你自己的邮箱
FROM_EMAIL = "mailgun@salvius.org"
RECIPIENTS = ["gunthercx@gmail.com"]

bot = ChatBot(
    "Mailgun Example Bot",
    mailgun_from_address=FROM_EMAIL,
    mailgun_api_key=MAILGUN["CONSUMER_KEY"],
    mailgun_api_endpoint=MAILGUN["API_ENDPOINT"],
    mailgun_recipients=RECIPIENTS,
    input_adapter="chatterbot.input.Mailgun",
    output_adapter="chatterbot.output.Mailgun",
    storage_adapter="chatterbot.storage.JsonFileStorageAdapter",
    database="../database.db"
)

# 简单的邮件回复
response = bot.get_response("How are you?")
print("Check your inbox at ", RECIPIENTS)

#### 一个中文的例子

注意chatterbot，中文聊天机器人的场景下一定要用python3.X，用python2.7会有编码问题。

In [20]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

#手动设置一些语料
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
 
 
Chinese_bot = ChatBot("Training demo")
#Chinese_bot.set_trainer(ListTrainer)
Chinese_bot=[
    '你好',
    '你好',
    '有什么能帮你的？',
    '想买数据科学的课程',
    '具体是数据科学哪块呢？',
    '机器学习',
]
 
trainer=ListTrainer(chatbot)
trainer.train(Chinese_bot)

# 测试一下
question = '你好'
print(question)
response = chatbot.get_response(question)
print(response)

print("\n")

question = '请问哪里能买数据科学的课程'
print(question)
response = chatbot.get_response(question)
print(response)

List Trainer: [####################] 100%

INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: 你好 1.0
INFO:chatterbot.chatterbot:Using "你好" as a close match to "你好" with a confidence of 1.0
INFO:chatterbot.chatterbot:Selecting response from 2 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 2 options.
INFO:chatterbot.chatterbot:Response selected. Using "你好"
INFO:chatterbot.chatterbot:BestMatch selected "你好" as a response with a confidence of 1.0
INFO:chatterbot.chatterbot:Adding "你好" as a response to "What time is it?"
INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Using "请问哪里能买数据科学的课程" as a close match to "请问哪里能买数据科学的课程" with a confidence of 0



你好
你好


请问哪里能买数据科学的课程


INFO:chatterbot.chatterbot:No responses found. Generating alternate response list.
INFO:chatterbot.chatterbot:No known response to the input was found. Selecting a random response.
INFO:chatterbot.chatterbot:BestMatch selected "Why would I feel sad? I don't understand." as a response with a confidence of 0
INFO:chatterbot.chatterbot:Adding "请问哪里能买数据科学的课程" as a response to "你好"


Why would I feel sad? I don't understand.


In [ ]:
!pip install chatterbot

#### 利用已经提供好的小中文语料库

In [51]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
 
chatbot = ChatBot("ChineseChatBot")
#chatbot.set_trainer(ChatterBotCorpusTrainer)
trainer=ChatterBotCorpusTrainer(chatbot)
 
# 使用中文语料库训练它
trainer.train("/content/drive/My Drive/VQA/chinese")
 
# 开始对话
while True:
    print(chatbot.get_response(input(">")))